In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from scipy.interpolate import interp1d
import pickle
from climateforcing.utils import mkdir_p

from fair21.constants.gases import molwt, burden_per_emission, lifetime, gas_list
from fair21.defaults.gases import (
    partition_fraction,
    pre_industrial_concentration,
    natural_emissions_adjustment,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    iirf_horizon
)
from fair21.gas_cycle import (
    calculate_g,
    calculate_alpha,
    step_concentration_1box,
    step_concentration_co2
)

In [ ]:
# would be awesome to now convert this to csv
with open("../data/output/rcmip-fair21-ssp245-concentrations.pkl","rb") as filein:
    concentration = pickle.load(filein)

In [ ]:
# interpolate all to a monthly timestep
t_new = np.arange(1750+1/24, 2501, 1/12)
concentration_monthly = {}

for specie in concentration.keys():
    f = interp1d(np.arange(1750.5, 2501), concentration[specie], fill_value="extrapolate")
    concentration_monthly[specie] = f(t_new)
    
f = interp1d(np.arange(1750.5, 2501), concentration['CO2'], fill_value='extrapolate')
concentration_co2_halfyear = f(np.arange(1750.25, 2501, 0.5))

In [ ]:
# grab some emissions
emissions_rcmip = {}
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for gas in gas_list:
    gas_rcmip_name = gas.replace("-", "")
    emissions_rcmip[gas] = df.loc[
        (df['Scenario']=='ssp245') & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
    ].interpolate(axis=1).values.squeeze()

# CO2 and N2O units need to behave
emissions_rcmip["CO2"] = emissions_rcmip["CO2"] / 1000
emissions_rcmip["N2O"] = emissions_rcmip["N2O"] / 1000

In [ ]:
# grab indicative temperature projections
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp245-mean-temperature.csv')
ssp245_temperature_rfmip = df['temperature'].values

t_new = np.arange(1750+1/24, 2501, 1/12)
f = interp1d(np.arange(1750.5, 2501), ssp245_temperature_rfmip, fill_value="extrapolate")
ssp245_temperature_monthly = f(t_new)

t_new = np.arange(1750.25, 2501, 0.5)
f = interp1d(np.arange(1750.5, 2501), ssp245_temperature_rfmip, fill_value="extrapolate")
ssp245_temperature_halfyear = f(t_new)

In [ ]:
# this will always be half a timestep out if we are not root-finding the new emissions, but we can live with it

def unstep_concentration(
    concentration,
    airborne_emissions_old,
    alpha_lifetime,
    lifetime,
    burden_per_emission,
    pre_industrial_concentration,
    timestep=1
):
    # these are, at least for the temperature independent model, invariant.
    decay_rate = timestep/(alpha_lifetime * lifetime)
    decay_factor = np.exp(-decay_rate)

    airborne_emissions_new = (concentration - pre_industrial_concentration) / burden_per_emission
    
    emissions = (
        airborne_emissions_new - (airborne_emissions_old * decay_factor)
    ) / (1 / decay_rate * (1 - decay_factor) * timestep) 

    return emissions, airborne_emissions_new


def unstep_concentration_co2(
    concentration,
    gas_boxes_old,
    airborne_emissions_old,
    alpha_lifetime,
    lifetime,
    burden_per_emission,
    pre_industrial_concentration,
    partition_fraction=partition_fraction['CO2'],
    timestep=1
):
    
    decay_rate = timestep/(alpha_lifetime * lifetime)   # [1]
    decay_factor = np.exp(-decay_rate)  # [1]

    # [GtCO2] = [ppm] - [ppm] / [ppm/GtCO2]
    airborne_emissions_new = (concentration-pre_industrial_concentration)/burden_per_emission
    
    # [GtCO2/yr] = [GtCO2] - [GtCO2]*[1] / ([1] * [1] * [1] * [yr])
    emissions =   (( airborne_emissions_new - np.sum(gas_boxes_old*decay_factor)) / (np.sum( partition_fraction / decay_rate * ( 1. - decay_factor ) * timestep)))

    # [GtCO2] = [yr] * [GtCO2/yr] * [1] / [1] * [1] + [GtCO2] * [1]
    gas_boxes_new = timestep * emissions * partition_fraction * 1/decay_rate * ( 1. - decay_factor ) + gas_boxes_old * decay_factor
    
    return emissions, gas_boxes_new, airborne_emissions_new

In [ ]:
emissions = {}

for specie in concentration_monthly.keys():

    emissions[specie] = np.ones(751*12) * np.nan
    airborne_emissions = 0
    
    if specie in ("CO2", "CH4"):
        continue

    for i in range(751*12):
        emissions[specie][i], airborne_emissions = unstep_concentration(
            concentration_monthly[specie][i],
            airborne_emissions,
            alpha_lifetime=1,
            lifetime=lifetime[specie],
            burden_per_emission=burden_per_emission[specie],
            pre_industrial_concentration=pre_industrial_concentration[specie],
            timestep=1/12
        )

In [ ]:
cumulative_emissions = {}

for specie in ["CH4"]:
    cumulative_emissions[specie] = 0
    emissions[specie][0] = 0
    airborne_emissions = 0
    g0, g1 = calculate_g(lifetime[specie], partition_fraction[specie])
    
    for i in range(751*12):
        
        alpha_lifetime = calculate_alpha(
            cumulative_emissions[specie],
            airborne_emissions,
            ssp245_temperature_monthly[i],
            iirf_0[specie],
            iirf_cumulative[specie],
            iirf_temperature[specie],
            iirf_airborne[specie],
            g0,
            g1
        )
        
        print(alpha_lifetime)
        
        emissions[specie][i], airborne_emissions = unstep_concentration(
            concentration_monthly[specie][i],
            airborne_emissions,
            alpha_lifetime=alpha_lifetime,
            lifetime=lifetime[specie],
            burden_per_emission=burden_per_emission[specie],
            pre_industrial_concentration=pre_industrial_concentration[specie],
            timestep=1/12
        )
        
        cumulative_emissions[specie] = np.sum(emissions[specie][:i] * 1/12)
    cumulative_emissions[specie] = np.cumsum(emissions[specie] * 1/12)

In [ ]:
for specie in ["CO2"]:
    cumulative_emissions[specie] = 0
    gas_boxes=np.zeros(4)
    emissions[specie][0] = 0
    airborne_emissions = 0
    g0, g1 = calculate_g(lifetime[specie], partition_fraction[specie])
    
    for i in range(751*12):
        
        alpha_lifetime = calculate_alpha(
            cumulative_emissions[specie],
            airborne_emissions,
            ssp245_temperature_monthly[i],
            iirf_0[specie],
            iirf_cumulative[specie],
            iirf_temperature[specie],
            iirf_airborne[specie],
            g0,
            g1
        )
        
        emissions[specie][i], gas_boxes, airborne_emissions = unstep_concentration_co2(
            concentration_monthly[specie][i],
            gas_boxes,
            airborne_emissions,
            alpha_lifetime=alpha_lifetime,
            lifetime=lifetime[specie],
            burden_per_emission=burden_per_emission[specie],
            pre_industrial_concentration=pre_industrial_concentration[specie],
            partition_fraction=partition_fraction[specie],
            timestep=1/12
        )
        
        cumulative_emissions[specie] = np.sum(emissions[specie][:i] * 1/12)
        print(emissions[specie][i])
    cumulative_emissions[specie] = np.cumsum(emissions[specie] * 1/12)

In [ ]:
emissions_co2_halfyear = np.zeros(751*2)
for specie in ["CO2"]:
    cumulative_emissions_co2_halfyear = 0
    gas_boxes=np.zeros(4)
    emissions_co2_halfyear[0] = 0
    airborne_emissions = 0
    g0, g1 = calculate_g(lifetime[specie], partition_fraction[specie])
    
    for i in range(751*2):
        
        alpha_lifetime = calculate_alpha(
            cumulative_emissions_co2_halfyear,
            airborne_emissions,
            ssp245_temperature_halfyear[i],
            iirf_0[specie],
            iirf_cumulative[specie],
            iirf_temperature[specie],
            iirf_airborne[specie],
            g0,
            g1
        )
        
        emissions_co2_halfyear[i], gas_boxes, airborne_emissions = unstep_concentration_co2(
            concentration_co2_halfyear[i],
            gas_boxes,
            airborne_emissions,
            alpha_lifetime=alpha_lifetime,
            lifetime=lifetime[specie],
            burden_per_emission=burden_per_emission[specie],
            pre_industrial_concentration=pre_industrial_concentration[specie],
            partition_fraction=partition_fraction[specie],
            timestep=0.5
        )
        
        cumulative_emissions_co2_halfyear = np.sum(emissions_co2_halfyear[:i] * 0.5)

    cumulative_emissions_co2_halfyear = np.cumsum(emissions_co2_halfyear * 0.5)

In [ ]:
pl.plot(concentration_co2_halfyear)

In [ ]:
pl.plot(np.arange(1750.25, 2501, 0.5), emissions_co2_halfyear)
pl.plot(np.arange(1750.5, 2501), emissions_rcmip['CO2'])

In [ ]:
fig, ax = pl.subplots(6, 8, figsize=(16,16))
for igas, gas in enumerate(gas_list):
    iy = igas % 8
    ix = igas // 8
    ax[ix, iy].plot(np.arange(1750.5, 2501), emissions_rcmip[gas], label='RCMIP')
    ax[ix, iy].plot(np.arange(1750+1/24, 2501, 1/12), emissions[gas], label='Back-calculated')
    ax[ix, iy].set_title(gas)
fig.tight_layout()

In [ ]:
emissions = {}

for specie in concentration.keys():

    emissions[specie] = np.ones(751) * np.nan
    airborne_emissions = 0
    
    if specie in ("CO2", "CH4"):
        continue

    for i in range(751):
        emissions[specie][i], airborne_emissions = unstep_concentration(
            concentration[specie][i],
            airborne_emissions_old = airborne_emissions,
            alpha_lifetime=1,
            lifetime=lifetime[specie],
            burden_per_emission=burden_per_emission[specie],
            pre_industrial_concentration=pre_industrial_concentration[specie],
            timestep=1
        )
        
        
for specie in ["CH4"]:
    cumulative_emissions[specie] = 0
    emissions[specie][0] = 0
    airborne_emissions = 0
    g0, g1 = calculate_g(lifetime[specie], partition_fraction[specie])
    
    for i in range(751):
        
        alpha_lifetime = calculate_alpha(
            cumulative_emissions[specie],
            airborne_emissions,
            ssp245_temperature_rfmip[i],
            iirf_0[specie],
            iirf_cumulative[specie],
            iirf_temperature[specie],
            iirf_airborne[specie],
            g0,
            g1
        )
        
        emissions[specie][i], airborne_emissions = unstep_concentration(
            concentration[specie][i],
            airborne_emissions,
            alpha_lifetime=alpha_lifetime,
            lifetime=lifetime[specie],
            burden_per_emission=burden_per_emission[specie],
            pre_industrial_concentration=pre_industrial_concentration[specie],
            timestep=1
        )
        
        cumulative_emissions[specie] = np.sum(emissions[specie][:i])
    cumulative_emissions[specie] = np.cumsum(emissions[specie])

for specie in ["CO2"]:
    cumulative_emissions[specie] = 0
    gas_boxes=np.zeros(4)
    emissions[specie][0] = 0
    airborne_emissions = 0
    g0, g1 = calculate_g(lifetime[specie], partition_fraction[specie])
    
    for i in range(751):
        
        alpha_lifetime = calculate_alpha(
            cumulative_emissions[specie],
            airborne_emissions,
            ssp245_temperature_rfmip[i],
            iirf_0[specie],
            iirf_cumulative[specie],
            iirf_temperature[specie],
            iirf_airborne[specie],
            g0,
            g1
        )
        
        print(alpha_lifetime)
        
        
        emissions[specie][i], gas_boxes, airborne_emissions = unstep_concentration_co2(
            concentration[specie][i],
            gas_boxes,
            airborne_emissions,
            alpha_lifetime=alpha_lifetime,
            lifetime=lifetime[specie],
            burden_per_emission=burden_per_emission[specie],
            pre_industrial_concentration=pre_industrial_concentration[specie],
            partition_fraction=partition_fraction[specie],
            timestep=1
        )
        #print(emissions[specie][i])
        cumulative_emissions[specie] = np.sum(emissions[specie][:i])
    cumulative_emissions[specie] = np.cumsum(emissions[specie])

In [ ]:
fig, ax = pl.subplots(6, 8, figsize=(16,16))
for igas, gas in enumerate(gas_list):
    iy = igas % 8
    ix = igas // 8
    ax[ix, iy].plot(np.arange(1750.5, 2501), emissions_rcmip[gas], label='RCMIP')
    ax[ix, iy].plot(np.arange(1750.5, 2501, 1), emissions[gas], label='Back-calculated')
    ax[ix, iy].set_title(gas)
fig.tight_layout()